In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('pyspark-examples') \
                    .getOrCreate()

In [4]:
df = spark.read.json("people.json")

df.show()
df.printSchema()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  25|  Anand|
|  40|Karthik|
|  50|Debasis|
|  28| Swathi|
|  30|   Mary|
|  45|  Sunil|
|  20|Manisha|
+----+-------+

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [5]:
df.createOrReplaceTempView("people")

In [7]:
teenagers = spark.sql("SELECT name, age FROM people WHERE age >= 13 AND age <= 19")

In [9]:
teenagers.show(truncate = False)

+------+---+
|name  |age|
+------+---+
|Justin|19 |
+------+---+



Find the count of teenagers (age <= 19) and adult.

In [19]:
spark.sql("SELECT count(name) as count, if(age <= 19, 'Teenager','Adults') as age_group FROM people where age is not Null group by (age_group)").show(truncate = False)

+-----+---------+
|count|age_group|
+-----+---------+
|1    |Teenager |
|8    |Adults   |
+-----+---------+



Using Case Statements

In [33]:
spark.sql("SELECT count(name) as count, case when age < 13 then 'Kids' \
		   when (age >= 13) and (age <= 19) then 'Teenagers' \
           when (age > 19) and (age <= 30) then 'Youngsters' \
           when (age > 30) and (age < 50) then 'MiddleAgers' \
           else 'Seniors' END as age_group \
           FROM people where age is not Null \
           group by (age_group)").show(truncate = False)

+-----+-----------+
|count|age_group  |
+-----+-----------+
|5    |Youngsters |
|1    |Seniors    |
|1    |Teenagers  |
|2    |MiddleAgers|
+-----+-----------+

